# Using Threshold - Based Source Detection and Confusion Matrix
This notebook provides an example of how to run the astropy - based source detection on a fits file. This also demonstrates how to generate a confusion matrix based on the results of the source detection.

In [21]:
import astropy
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel
from astropy.stats import sigma_clipped_stats,gaussian_fwhm_to_sigma,gaussian_sigma_to_fwhm
from astropy.table import Table,Column,Row,vstack,setdiff,join
from astropy.nddata import Cutout2D,NDData
import astropy.units as u
import os 
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval,simple_norm
from astropy.wcs.utils import skycoord_to_pixel, pixel_to_skycoord

import photutils
from photutils.datasets import make_gaussian_sources_image

import numpy as np
import itertools
import copy

import matplotlib
from astropy.visualization import ZScaleInterval,simple_norm
zscale = ZScaleInterval()
import pickle
import numpy as np

In [22]:
import diffimageml
from diffimageml import fakeplanting, util

# Make a FitsImage class object with the desired image

In [23]:
example_file = "../test_data/sky_image_1.fits.fz"
FitsImageExample = diffimageml.FitsImage(example_file)

# Run source detection

In [27]:
sources = FitsImageExample.detect_sources()
print (sources[0])

Set OBSGEO-B to   -31.272933 from OBSGEO-[XYZ].
Set OBSGEO-H to     1164.993 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


<photutils.segmentation.properties.SourceProperties>
label: 1
centroid (x, y): (833.3810, 31.6253)
sky_centroid: None


# Confusion Matrix example
   The confusion matrix requires a fakeplanter object, which needs to have images with fakes planted.

In [40]:
#Load examle with fakes
diffim = "../test_data/diff_pydia_1_fakegrid.fits"
search_im = "../test_data/sky_image_1.fits.fz"
fake_sn = diffimageml.FakePlanter(diffim , searchim_fitsfilename = search_im)

Set OBSGEO-B to   -31.272933 from OBSGEO-[XYZ].
Set OBSGEO-H to     1164.993 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [41]:
matrix = fake_sn.confusion_matrix()

Gaia catalog ../test_data/sky_image_1_GaiaCat.ecsv exists. 
Reading without fetching.


/Users/Peter/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:748: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/Peter/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:748: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/Peter/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:748: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/Peter/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:748: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/Peter/Library/Python/3.7/lib/python/site-packages/numpy/core/fromnumeric.py:748: UserWarning: Warning: 'partition' wi

 x    y   detect        mag        
---- ---- ------ ------------------
 100  100      1  16.58507718268365
 100  200      1 16.589656194870322
 100  300      1  16.58538909278126
 100  400      1 16.581846608179593
 100  500      1  16.57693919196567
 100  600      1 16.586929393934767
 100  700      1 16.582391736322354
 100  800      1 16.585706995904182
 100  900      1  16.58519974811047
 100 1000      1  16.58428879954485
 ...  ...    ...                ...
2100 1100      1  16.58363471997946
2100 1200      1  16.58780784031031
2100 1300      1 16.577510113189874
2100 1400      1  16.58439965497042
2100 1500      1 16.590504001711274
2100 1600      1  16.58348138471257
2100 1700      1 16.586621399740235
2100 1800      1 16.582318657498973
2100 1900      1  16.59046309215767
2100 2000      1  16.58806166443344
2100 2100      1 16.585429412644338
Length = 440 rows


/Users/Peter/Library/Python/3.7/lib/python/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other


The output matrix is a list of astropy tables containing the locations and magnitudes for the sources in the given categories. To simply display the resulting 2x2 matrix, use the following.

In [42]:
print (len(matrix[0]) , len(matrix[1]))
print (len(matrix[2]) , 0) ##No true negatives

440 1
588 0


In this case we recover 440 True positives, one False negative and 588 false positives. We can also make a confusion matrix that imposes magnitude limits. For this example, the fake sources have magnitudes around 16.5, so if we limit the magnitudes to be between 18 and 22, we filter out all the true positives.

In [43]:
matrix = fake_sn.confusion_matrix(low_mag_lim = 18 , high_mag_lim = 22)

/Users/Peter/Library/Python/3.7/lib/python/site-packages/astropy/table/table.py:3183: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other


In [44]:
print (len(matrix[0]) , len(matrix[1]))
print (len(matrix[2]) , 0) ##No true negatives

0 0
1 0
